In [2]:
import pypint

You are using Pint version 2017-04-13 and pypint 1.0

## Transient reachability analysis

The main features of pint relate to the transient reachability analysis, for which efficient over- and under-approximation have been designed.

The reachability properties are specfied in term of a *goal*. We first detail the different possible specifications for such a goal, and then illustrate various analyses of its reachability.

### Goal specification

In its simplest form, a goal is the local state of one automaton, for instance `g=1`. The goal is reached as soon as a state where `g=1` is reached. Note that there is no assumption on the stability of this state, this is why we refer to the *transient* reachability analysis.

Besides the single local state, goal can be
* a (sub-)state, specified either as by a dict-like object or string representation of the (sub)-state
* a sequence of (sub-)state: for instance a `Goal("a=1","b=1")` is reached if one can reach a state where `a=1` and from which can be reached a state where `b=1`
* a disjunction of goals: `Goal("a=1")|Goal("c=1")` is reached if one can reach a state where either `a=1` or `c=1`.

In [2]:
from pypint import Goal # avoid typing pypint
simple = Goal("g=1")    # simple goal
simple = Goal(g=1) # equivalent notation
substate = Goal({"a": 1, "b": 1})   # reach a state where both a=1 and b=1
seq = Goal({"a": 1, "b": 1}, {"c": 1}) # reach a state where a=1 and b=1 and from c=1 is reachable
seq = Goal("a=1,b=1", "c=1") # equivalent to above
alt = Goal("a=1", "b=1") | Goal("c=1") # either reach a state where a=1 and then a state where b=1, 
                                       # or reach a state where c=1

Note that if the goal consists of a single argument (such as a simple local state), there is no need to explicitly instanciates this class: `method(Goal("g=1"))` is equivalent to `method("g=1")`.

### Static reachability analysis

In [3]:
mapk = pypint.load("http://ginsim.org/sites/default/files/MAPK_large_19june2013.zginml")
mapk.summary()

Downloading 'http://ginsim.org/sites/default/files/MAPK_large_19june2013.zginml' to 'gen/pintx6ie2p7pMAPK_large_19june2013.zginml'

Source file is in zginml format, importing with logicalmodel

Invoking GINsim...

Simplifying model...

/home/pauleve/rech/pint/docs/src/doc/gen/pintjoscjnk5MAPK_large_19june2013.an

25 state(s) have been registered: initState_23, initState_13, initState_7, initState_9, initState_3, initState_17, initState_12, initState_4, initState_11, initState_19, initState_15, initState_16, initState_8, initState_20, initState_2, initState_24, initState_21, initState_10, initState_5, initState_22, initState_25, initState_6, initState_14, initState_18, initState_1

{'max_local_states': 2,
 'nb_automata': 53,
 'nb_local_states': 106,
 'nb_states': 9007199254740992,
 'nb_transitions': 177}

In [4]:
mapk.having("initState_1").reachability("Proliferation=1")

True

In [6]:
mapk.having("initState_10").reachability("Proliferation=1")

Approximations are inconclusive, fallback to exact model-checking with `its`

True

In [7]:
mapk.having("initState_10").reachability("Proliferation=1", fallback=None)

pypint.types.Inconc

In [ ]:
mapk.having("initState_1").reachability("Proliferation=1", tool="nusmv")

### Identification of mutations to control goal reachability

In [13]:
wt = pypint.load("models/invasion_pathways.an")
wt.initial_state["ECMicroenv"] = 1
goal = pypint.Goal("Apoptosis=1") | pypint.Goal("CellCycleArrest=1")
mutations = wt.oneshot_mutations_for_cut(goal, maxsize=3, exclude={"ECMicroenv"})
mutations

Source file is in Automata Network (an) format

This computation is an *under-approximation*: returned mutations are all valid, but they may be non-minimal, and some solutions may be missed.

Limiting solutions to mutations of at most 3 automata. Use `maxsize` argument to change.

[{'AKT1': 1},
 {'CDH1': 1, 'NICD': 0},
 {'CDH2': 1, 'NICD': 0},
 {'CTNNB1': 0, 'NICD': 0},
 {'DKK1': 1, 'NICD': 0},
 {'AKT2': 1, 'ERK': 1, 'ZEB2': 0},
 {'AKT2': 1, 'ERK': 1, 'p63': 1},
 {'AKT2': 1, 'ZEB2': 0, 'p21': 0},
 {'ERK': 1, 'SNAI1': 1, 'ZEB2': 0},
 {'ERK': 1, 'SNAI2': 1, 'ZEB2': 0},
 {'ERK': 1, 'SNAI2': 1, 'p63': 1},
 {'ERK': 1, 'ZEB1': 1, 'ZEB2': 0},
 {'ERK': 1, 'ZEB1': 1, 'p53': 1},
 {'ERK': 1, 'ZEB1': 1, 'p63': 1},
 {'ERK': 1, 'ZEB2': 0, 'p53': 0},
 {'ERK': 1, 'miR200': 0, 'p63': 1},
 {'NICD': 0, 'SNAI2': 1, 'TWIST1': 0},
 {'NICD': 0, 'TWIST1': 0, 'p53': 0},
 {'SNAI2': 1, 'ZEB2': 0, 'p21': 0},
 {'ZEB2': 0, 'p21': 0, 'p53': 0}]

In [16]:
wt.reachability(goal)

True

In [14]:
wt.lock(mutations[6]).reachability(goal)

False

### Cut sets of paths to goal

In [ ]:
wt.reachability("Apoptosis=1")

In [ ]:
wt.cutsets("Apoptosis=1",maxsize=3)

In [ ]:
wt.disable(p53=1).reachability("Apoptosis=1")

In [ ]:
wt.cutsets("Apoptosis=1",maxsize=3,exclude_initial_state=False)

In [ ]:
wt.disable(p53=0).reachability("Apoptosis=1")

### Identification of bifurcation transitions

In [ ]:
wt.bifurcations("Apoptosis=1")

### Goal-oriented model reduction

Pint can statically detect part of transitions that do not take part in minimal paths to the goal reachability. A path is minimal if and only it contains no cycle and all the transitions are causally related.
These transitions can then be removed from the model while preserving all the minimal paths to the goal and potentially reducing the reachable state grah significantly.

The following command loads a model of 53 automata and 173 local transitions.
The reduction for reachability of active Apoptosis from the state where DNA_damage is on leads to an automata network with only 69 local transitions. In this case, the model-checking with NuSMV is tractable only with the reduced model.

In [5]:
mapk53 = pypint.load("models/mapk53.an")
len(mapk53.automata), len(mapk53.local_transitions)

Source file is in Automata Network (an) format

(53, 173)

In [11]:
red = mapk53.having(DNA_damage=1).reduce_for_goal(Apoptosis=1)
red.save_as("gen/mapk53-apoptosis.smv")  # export for later NuSMV model checking
len(red.local_transitions)

/home/pauleve/rech/pint/docs/src/doc/gen/pint890dwkid.an

69

### Related functions and classes